In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import torch
from torch.nn import BCEWithLogitsLoss, BCELoss
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix, f1_score, accuracy_score
import pickle
from transformers import *
from tqdm import tqdm, trange
from ast import literal_eval
from sklearn import metrics
import random
import matplotlib.pyplot as plt
from numba import jit, cuda
from transformers import BertTokenizer
from sklearn.metrics import multilabel_confusion_matrix
import pickle
#import cuda


def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    '''
    Compute the Hamming score (a.k.a. label-based accuracy) for the multi-label case
    http://stackoverflow.com/q/32239577/395857
    '''
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        #print('\nset_true: {0}'.format(set_true))
        #print('set_pred: {0}'.format(set_pred))
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)) )
        #print('tmp_a: {0}'.format(tmp_a))
        acc_list.append(tmp_a)
    return np.mean(acc_list)


#using cpu as our device
device = torch.device("cuda")
# If there's a GPU available...
print(torch.cuda.is_available())



# Set the seed value all over the place to make this reproducible.
seed_val = 5
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed(seed_val)

#pd.set_option('display.max_columns', None)


# Load the dataset into a pandas dataframe.
dff = pd.read_csv("../input/frames-sentence-level/actual_dataset.csv", header=None, names=['text','code', 'one_hot_labels'])
#print(dff)

#train_test_validate split (60,20,20)
train, validate, test = \
              np.split(dff.sample(frac=1, random_state=42),
                       [int(.6*len(dff)), int(.8*len(dff))])

#print(train)
#print(validate)
#print(test)
train_validate_set = [train,validate]
df = pd.concat(train_validate_set,ignore_index='true')
df = pd.DataFrame(df, columns=['text', 'code', 'one_hot_labels'])
#df = dff
#print(df)
#pd.set_option('display.max_rows', None)
#print(test['code'].value_counts().nlargest(10)[test['code'].value_counts().nlargest(10) ==test['code'].value_counts().nlargest(10)])
num_labels = 15
#print('Unique comments: ', df.text.nunique() == df.shape[0])
'''
word_len_list = []
#print('average sentence length: ', df.text.str.split().str.len().mean())
length = df.text.str.split().str.len()

for i in length:
    print(i)
    word_len_list.append(i)
plt.hist(word_len_list, bins=500)
plt.xlabel('Number of Words')
plt.xticks(np.arange(0,200, step=5))
plt.margins(0)
plt.ylabel('Number of Sentences')
plt.title('Distribution of Words')
plt.show()
#print('stdev sentence length: ', df.text.str.split().str.len().std())
#print('Null values: ', df.isnull().values.any())
'''
#df["text"] = df["text"].apply(lambda x: list(map(str, x)))
#df["one_hot_labels"] = df["one_hot_labels"]
labels = list(df.one_hot_labels.values)


#Rearranging the label data in the dataframe to feed in the bert encodings
labels_re = []
labels_re_re = []
temp = []
for a in labels:
    labels_re = []
    for aa in a:

        if aa != " ":
            #print(aa)
            #print('###')
            labels_re.append(int(aa))
            #print(labels_re)
    labels_re_re.append(labels_re)
    temp = labels_re_re
    #print(labels_re_re)
#print(temp)

labels = temp
df['one_hot_labels'] = labels
new_df = df[['text', 'one_hot_labels']]

df = new_df
#print(df)

labels = list(df.one_hot_labels.values)
comments = list(df.text.values)


max_length = 128
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True) # tokenizer
encodings = tokenizer.batch_encode_plus(comments,max_length=max_length,pad_to_max_length=True) # tokenizer's encoding method
#print('tokenizer outputs: ', encodings.keys())

input_ids = encodings['input_ids'] # tokenized and encoded sentences
print('input_ids', len(input_ids))
token_type_ids = encodings['token_type_ids'] # token type ids
attention_masks = encodings['attention_mask'] # attention masks

# Identifying indices of 'one_hot_labels' entries that only occur once - this will allow us to stratify split our training data later
label_counts = df.one_hot_labels.astype(str).value_counts()
#print('label counts', label_counts)
one_freq = label_counts[label_counts==1].keys()
#print('one freq', one_freq)
one_freq_idxs = sorted(list(df[df.one_hot_labels.astype(str).isin(one_freq)].index), reverse=True)
#print('df label indices with only one instance: ', one_freq_idxs)

# Gathering single instance inputs to force into the training set after stratified split
one_freq_input_ids = [input_ids.pop(i) for i in one_freq_idxs]
#print(one_freq_input_ids)
one_freq_token_types = [token_type_ids.pop(i) for i in one_freq_idxs]
one_freq_attention_masks = [attention_masks.pop(i) for i in one_freq_idxs]
one_freq_labels = [labels.pop(i) for i in one_freq_idxs]

# Use train_test_split to split our data into train and validation sets

train_inputs, validation_inputs, train_labels, validation_labels, train_token_types, validation_token_types, train_masks, validation_masks = train_test_split(input_ids, labels, token_type_ids,attention_masks, test_size=0.25, shuffle = False) 
train_inputs.extend(one_freq_input_ids)
train_labels.extend(one_freq_labels)
train_masks.extend(one_freq_attention_masks)
train_token_types.extend(one_freq_token_types)

#print(train_labels.value_counts().nsmallest(10)[train_labels.value_counts().nsmallest(10) == train_labels.value_counts().nsmallest(10)])


#print('train labels', len(train_labels))
#print('val labels', len(validation_labels))
#print('plain train inputs',train_inputs)
#print('plain train labels', train_labels)

# Convert all of our data into torch tensors, the required datatype for our model
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
train_token_types = torch.tensor(train_token_types)

#print('tensor train inputs', train_inputs)
#print('tensor train labels', train_labels)

validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)
validation_token_types = torch.tensor(validation_token_types)

# Select a batch size for training. For fine-tuning with XLNet, the authors recommend a batch size of 32, 48, or 128. We will use 32 here to avoid memory issues.
batch_size = 32

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop,
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels, train_token_types)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels, validation_token_types)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

torch.save(validation_dataloader,'validation_data_loader')
torch.save(train_dataloader,'train_data_loader')

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)
model.cuda()

# setting custom optimization parameters. You may implement a scheduler here as well.
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]
#lr = np.arange(2e-5, 6e-5, 0.000002)
#lr = list(lr)
#learning_rate_list = []
#loss_list=[]
#print('lr', len(lr))

#learning_rate_list.append(l)

optimizer = AdamW(optimizer_grouped_parameters,lr=5.8e-5,correct_bias=True)
#optimizer = AdamW(model.parameters(),lr=2e-5)  # Default optimization

# Store our loss and accuracy for plotting
train_loss_set = []

#Number of training epochs (authors recommend between 2 and 4)
epochs = 3


# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):

    # Training

    # Set our model to training mode (as opposed to evaluation mode)
    model.train()

    # Tracking variables
    tr_loss = 0  # running loss
    nb_tr_examples, nb_tr_steps = 0, 0

    # Train the data for one epoch
    for step, batch in enumerate(train_dataloader):
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels, b_token_types = batch
        # Clear out the gradients (by default they accumulate)
        optimizer.zero_grad()

        # Forward pass for multilabel classification
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

        #print('outputs', outputs)
        logits = outputs[0]

        loss_func = BCEWithLogitsLoss()
        loss = loss_func(logits.view(-1, num_labels),
                         b_labels.type_as(logits).view(-1, num_labels))    # convert labels to float for calculation

        # Backward pass
        loss.backward()

        # Update parameters and take a step using the computed gradient
        optimizer.step()

        # Update tracking variables
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    print("Train loss: {}".format(tr_loss/nb_tr_steps))



    ###############################################################################

    # Validation

    # Put model in evaluation mode to evaluate loss on the validation set
    model.eval()

    # Variables to gather full output
    logit_preds, true_labels, pred_labels, tokenized_texts = [], [], [], []

    # Predict
    for i, batch in enumerate(validation_dataloader):
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels, b_token_types = batch
        with torch.no_grad():
            # Forward pass
            outs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
            b_logit_pred = outs[0]
            pred_label = torch.sigmoid(b_logit_pred)

            b_logit_pred = b_logit_pred.detach().cpu().numpy()
            pred_label = pred_label.to('cpu').numpy()
            b_labels = b_labels.to('cpu').numpy()

        tokenized_texts.append(b_input_ids)
        logit_preds.append(b_logit_pred)
        true_labels.append(b_labels)
        pred_labels.append(pred_label)

    # Flatten outputs
    pred_labels = [item for sublist in pred_labels for item in sublist]
    #print(pred_labels)
    true_labels = [item for sublist in true_labels for item in sublist]
    #print(true_labels)
    # Calculate Accuracy
    threshold = 0.50
    pred_bools = [pl > threshold for pl in pred_labels]
    #print(pred_bools)
    true_bools = [tl == 1 for tl in true_labels]
    #print(true_bools)
    val_f1_accuracy = f1_score(true_bools, pred_bools, average='micro') * 100
    val_flat_accuracy = accuracy_score(true_bools, pred_bools) * 100

    print('F1 Validation Accuracy: ', val_f1_accuracy)
    print('Flat Validation Accuracy: ', val_flat_accuracy)

pickle.dump(model, open('model_pickle.sav', 'wb'))


#test begins

print(test)
# Gathering input data
test_labels = list(test.one_hot_labels.values)

#Rearranging the label data in the dataframe to feed in the bert encodings
test_labels_re = []
test_labels_re_re = []
test_temp = []
for a in test_labels:
    test_labels_re = []
    for aa in a:

        if aa != " ":
            #print(aa)
            #print('###')
            test_labels_re.append(int(aa))
            #print(labels_re)
    test_labels_re_re.append(test_labels_re)
    test_temp = test_labels_re_re
    #print(labels_re_re)
#print(temp)

test_labels = test_temp
test['one_hot_labels'] = test_labels
new_test = test[['text', 'one_hot_labels']]
print(new_test)
new_test = test

test_labels = list(test.one_hot_labels.values)
test_comments = list(test.text.values)



# Encoding input data
test_encodings = tokenizer.batch_encode_plus(test_comments,max_length=max_length,pad_to_max_length=True, truncation=True)
test_input_ids = test_encodings['input_ids']
print('test_input_ids', len(test_input_ids))
test_token_type_ids = test_encodings['token_type_ids']
test_attention_masks = test_encodings['attention_mask']

# Make tensors out of data
test_inputs = torch.tensor(test_input_ids)
test_labels = torch.tensor(test_labels)
test_masks = torch.tensor(test_attention_masks)
test_token_types = torch.tensor(test_token_type_ids)
# Create test dataloader
test_data = TensorDataset(test_inputs, test_masks, test_labels, test_token_types)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)
# Save test dataloader
torch.save(test_dataloader,'test_data_loader')

    



In [6]:
import pandas as pd
import numpy as np
import tensorflow as tf
import torch
from torch.nn import BCEWithLogitsLoss, BCELoss
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix, f1_score, accuracy_score, ConfusionMatrixDisplay
import pickle
from transformers import *
from tqdm import tqdm, trange
from ast import literal_eval
from sklearn import metrics
import random
import matplotlib.pyplot as plt
from numba import jit, cuda
from transformers import BertTokenizer
from sklearn.metrics import multilabel_confusion_matrix
import pickle
from operator import and_
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from scipy import interp
from itertools import cycle
from PIL import Image  
import PIL


def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    '''
    Compute the Hamming score (a.k.a. label-based accuracy) for the multi-label case
    http://stackoverflow.com/q/32239577/395857
    '''
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        #print('\nset_true: {0}'.format(set_true))
        #print('set_pred: {0}'.format(set_pred))
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)) )
        #print('tmp_a: {0}'.format(tmp_a))
        acc_list.append(tmp_a)
    return np.mean(acc_list)

#load the model from disk
model = pickle.load(open("../input/model/model_pickle.sav", 'rb'))

# Load the dataset into a pandas dataframe.
dff = pd.read_csv('../input/frames-sentence-level/actual_dataset.csv', header=None, names=['text','code', 'one_hot_labels'])
#print(dff)

#train_test_validate split (60,20,20)
train, validate, test = \
              np.split(dff.sample(frac=1, random_state=42),
                       [int(.6*len(dff)), int(.8*len(dff))])




#using cpu as our device
device = torch.device("cuda")

#test begins

# Gathering input data
test_labels = list(test.one_hot_labels.values)

#Rearranging the label data in the dataframe to feed in the bert encodings
test_labels_re = []
test_labels_re_re = []
test_temp = []
for a in test_labels:
    test_labels_re = []
    for aa in a:

        if aa != " ":
            #print(aa)
            #print('###')
            test_labels_re.append(int(aa))
            #print(labels_re)
    test_labels_re_re.append(test_labels_re)
    test_temp = test_labels_re_re
    #print(labels_re_re)
#print(temp)

test_labels = test_temp
test['one_hot_labels'] = test_labels
new_test = test[['text', 'one_hot_labels']]
#print(new_test)
test = new_test

test_labels = list(test.one_hot_labels.values)
test_comments = list(test.text.values)


#count_label = tf.reduce_sum(np.array(test_labels), axis=0)
#print(count_label)



tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True) # tokenizer


# Encoding input data
test_encodings = tokenizer.batch_encode_plus(test_comments,max_length=128,pad_to_max_length=True, truncation=True)
test_input_ids = test_encodings['input_ids']
#print('test_input_ids', len(test_input_ids))
test_token_type_ids = test_encodings['token_type_ids']
test_attention_masks = test_encodings['attention_mask']

# Make tensors out of data
test_inputs = torch.tensor(test_input_ids)
test_labels = torch.tensor(test_labels)
test_masks = torch.tensor(test_attention_masks)
test_token_types = torch.tensor(test_token_type_ids)
# Create test dataloader
test_data = TensorDataset(test_inputs, test_masks, test_labels, test_token_types)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=32)
# Save test dataloader
#torch.save(test_dataloader,'test_data_loader')
#Test
df = test

#Put model in evaluation mode to evaluate loss on the validation set
model.eval()

#track variables
logit_preds,true_labels,pred_labels,tokenized_texts = [],[],[],[]

# Predict
for i, batch in enumerate(test_dataloader):
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels, b_token_types = batch
  with torch.no_grad():
    # Forward pass
    outs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    b_logit_pred = outs[0]
    pred_label = torch.sigmoid(b_logit_pred)

    b_logit_pred = b_logit_pred.detach().cpu().numpy()
    pred_label = pred_label.to('cpu').numpy()
    b_labels = b_labels.to('cpu').numpy()

  tokenized_texts.append(b_input_ids)
  logit_preds.append(b_logit_pred)
  true_labels.append(b_labels)
  pred_labels.append(pred_label)

# Flatten outputs
tokenized_texts = [item for sublist in tokenized_texts for item in sublist]
pred_labels = [item for sublist in pred_labels for item in sublist]
true_labels = [item for sublist in true_labels for item in sublist]
# Converting flattened binary values to boolean values
true_bools = [tl==1 for tl in true_labels]

pred_bools = [pl>0.50 for pl in pred_labels] #boolean output after thresholding
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option("display.max_colwidth", -1)


true_bools_wrong , pred_bools_wrong = [],[]

for t in range(len(true_bools)):
    if str(true_bools[t]) != str(pred_bools[t]):
        #print(df.iloc[t])
        #print(pred_bools[t])
        true_bools_wrong.append(true_bools[t])
        pred_bools_wrong.append(pred_bools[t])

#print('Test F1 Accuracy: ', f1_score(true_bools_wrong, pred_bools_wrong,average='micro'))
#print('Test Flat Accuracy: ', accuracy_score(true_bools_wrong, pred_bools_wrong),'\n')
#print('Hamming score: {0}'.format(hamming_score(np.array(true_bools_wrong), np.array(pred_bools_wrong))))  

#clf_report = classification_report(true_bools,pred_bools,target_names=['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14'])
#pickle.dump(clf_report, open('classification_report.txt','wb')) #save report
print(multilabel_confusion_matrix(true_bools_wrong,pred_bools_wrong))
#print(clf_report)  

y = label_binarize(true_bools, classes=['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14'])
n_classes = y.shape[1]

z = label_binarize(pred_bools, classes=['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14'])


print(y)
print(z)
#print(true_bools)
#print(pred_bools)

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y[:,i], z[:,i], pos_label = 15)
    roc_auc[i] = auc(fpr[i], tpr[i])

#print(roc_auc)
print(fpr)
print('##########################')
print(tpr)
'''
# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y.ravel(), z.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

#Compute macro-average ROC curve and ROC area

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure(figsize=(12,8))
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['#00FFFF','#C0C0C0','#0000FF','#808080','#0000A0','#000000','#ADD8E6','#FFA500','#800080','#A52A2A','#FFFF00','#800000','#00FF00','#008000','#FF00FF'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) curve')
plt.legend(loc="lower right")
plt.show()
plt.savefig('ROC.jpg')
   


                
# Print and save classification report
print('Test F1 Accuracy: ', f1_score(true_bools, pred_bools,average='micro'))
print('Test F1 Macro:',  f1_score(true_bools, pred_bools,average='macro'))
print('Test Flat Accuracy: ', accuracy_score(true_bools, pred_bools),'\n')
print('Hamming score: {0}'.format(hamming_score(np.array(true_bools), np.array(pred_bools))))  

clf_report = classification_report(true_bools,pred_bools,target_names=['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14'])
#pickle.dump(clf_report, open('classification_report.txt','wb')) #save report
print(multilabel_confusion_matrix(true_bools,pred_bools))
#print(clf_report)


'''




/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:169: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


[[[12930  1169]
  [ 1366  1594]]

 [[16318   164]
  [  464   113]]

 [[15469   414]
  [  709   467]]

 [[15422   408]
  [  761   468]]

 [[10884  1614]
  [ 2036  2525]]

 [[12332  1093]
  [ 2485  1149]]

 [[14619   697]
  [  944   799]]

 [[15847   424]
  [  405   383]]

 [[14756   465]
  [ 1135   703]]

 [[13448  1186]
  [ 1665   760]]

 [[13548   890]
  [ 1982   639]]

 [[14180   923]
  [ 1193   763]]

 [[11136  1572]
  [ 1879  2472]]

 [[16556   142]
  [  280    81]]

 [[16714   108]
  [  216    21]]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 1 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 1 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
{0: array([0.        , 0.15715712, 1.        ]), 1: array([0.        , 0.01427356, 1.        ]), 2: array([0.        , 0.03901194, 1.        ]), 3: array([0.        , 0.07214436, 1.        ]), 4: array([0.        , 0.09625411, 1.        ]), 5: array(

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_ranking.py:813: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)


'\n# Compute micro-average ROC curve and ROC area\nfpr["micro"], tpr["micro"], _ = roc_curve(y.ravel(), z.ravel())\nroc_auc["micro"] = auc(fpr["micro"], tpr["micro"])\n\n#Compute macro-average ROC curve and ROC area\n\n# First aggregate all false positive rates\nall_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))\n\n# Then interpolate all ROC curves at this points\nmean_tpr = np.zeros_like(all_fpr)\nfor i in range(n_classes):\n    mean_tpr += interp(all_fpr, fpr[i], tpr[i])\n\n# Finally average it and compute AUC\nmean_tpr /= n_classes\n\nfpr["macro"] = all_fpr\ntpr["macro"] = mean_tpr\nroc_auc["macro"] = auc(fpr["macro"], tpr["macro"])\n\n# Plot all ROC curves\nplt.figure(figsize=(12,8))\nplt.plot(fpr["micro"], tpr["micro"],\n         label=\'micro-average ROC curve (area = {0:0.2f})\'\n               \'\'.format(roc_auc["micro"]),\n         color=\'deeppink\', linestyle=\':\', linewidth=4)\n\nplt.plot(fpr["macro"], tpr["macro"],\n         label=\'macro-average ROC